# Under-Representation Bias (w/ Synthetic Data)

This notebook recreates the finding that Equalized Odds constrained model can recover from under-representation bias.

### Setup

Please run the code block below to install the necessary packages (if needed).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve, auc
from collections import Counter

import fairlearn
from fairlearn.metrics import *
from fairlearn.reductions import *
import aif360

import copy, random

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Synthetic Dataset Generation

## Parameters (User Input)

In [2]:
'''

r is the proportion of training examples in the minority group, 

which means 1-r is proportion of examples in the majority group

eta is the probability of flipping the label

n is the number of training examples

beta is the probability of keeping a positively labeled example
from the minority class

'''
def get_params(r = 1/3, eta = 1/4, n = 2000, beta = 0.5):
    return r, eta, n, beta

r, eta, n, beta = get_params()

## True Label Generation

In [3]:
# create minority and majority groups
def get_cat_features(n, r):
    num_minority = int(r * n)
    num_majority = n - num_minority
    
    minority = np.zeros((num_minority, 1))
    majority = np.ones((num_majority, 1))
    
    cat_features = np.vstack((minority, majority))
    
    # shuffle so as to ensure randomness
    np.random.shuffle(cat_features)
    
    return cat_features

In [4]:
# simulate Bayes Optimal Classifiers for majority and minority groups
def get_bayes_optimal_labels(df_majority, df_minority):
    # format data
    X_maj = df_majority.iloc[:, :-1].values
    y_maj = df_majority.iloc[:, -1].values
    
    X_min = df_minority.iloc[:, :-1].values
    y_min = df_minority.iloc[:, -1].values

    classifier = LogisticRegression(random_state=42)
    
    # true labels
    
    classifier_majority = classifier.fit(X_maj, y_maj)
    y_pred_maj = classifier_majority.predict(X_maj).reshape(len(X_maj), 1)
    # print("Accuracy on Majority: ", accuracy_score(y_pred=y_pred_maj, y_true= y_maj))
    
    classifier_minority = classifier.fit(X_min, y_min)
    y_pred_min = classifier_minority.predict(X_min).reshape(len(X_min), 1)
    # print("Accuracy on Minority: ", accuracy_score(y_pred=y_pred_min, y_true= y_min))
    
    majority = np.hstack((X_maj, y_pred_maj))
    minority = np.hstack((X_min, y_pred_min))
    
    total = np.vstack((majority, minority))
    
    # shuffle so as to ensure randomness
    np.random.shuffle(total)
    
    df_true = pd.DataFrame(pd.DataFrame(total))
    df_true.columns = ['num1','num2','num3','cat','outcome']
    
    return df_true
    

In [5]:
# flip labels with probability eta
def flip_labels(df_synthetic, eta):
    labels = df_synthetic['outcome']
    
    for i in range(len(labels)):
        if random.uniform(0,1) <= eta:
            labels[i] = 1 if labels[i] == 0 else 1
    df_synthetic['outcome'] = labels
    
    return df_synthetic

In [75]:
# ensure equal proportion of positive examples across minority and majority
def equal_base_rates(df_majority, df_minority):
    base_rate_maj = df_majority['outcome'].value_counts()[0] / len(df_majority)
    base_rate_min = df_minority['outcome'].value_counts()[0] / len(df_minority)
    
    X_maj_pos = df_majority[df_majority['outcome'] == 1].iloc[:, :].values
    X_maj_neg = df_majority[df_majority['outcome'] == 0].iloc[:, :].values
    
    diff = round(base_rate_maj,4) - round(base_rate_min,4)
    
    count = 0
    
    if diff > 0:
        while(diff > 0.01):
            X_maj_neg = np.delete(X_maj_neg, 0, axis = 0)

            df_majority = pd.DataFrame(pd.DataFrame(np.vstack((X_maj_pos, X_maj_neg))))
            df_majority.columns = ['num1','num2','num3','cat','outcome']

            base_rate_maj = df_majority['outcome'].value_counts()[0] / len(df_majority)
            # print(len(df_majority))
            diff = round(base_rate_maj,4) - round(base_rate_min,4)
            count+=1

            # fail-safe
            if count > int(len(df_majority)/3): break
    else:
        diff = round(base_rate_min,4) - round(base_rate_maj,4) 
        while(diff > 0.01):
            X_maj_pos = np.delete(X_maj_pos, 0, axis = 0)

            df_majority = pd.DataFrame(pd.DataFrame(np.vstack((X_maj_pos, X_maj_neg))))
            df_majority.columns = ['num1','num2','num3','cat','outcome']

            base_rate_maj = df_majority['outcome'].value_counts()[0] / len(df_majority)
            diff = round(base_rate_min,4) - round(base_rate_maj,4) 
            count+=1

            # fail-safe
            if count > int(len(df_majority)/3): break
                
    total = np.vstack((df_majority, df_minority))
    
    # shuffle so as to ensure randomness
    np.random.shuffle(total)
                
    df_true = pd.DataFrame(pd.DataFrame(total))
    df_true.columns = ['num1','num2','num3','cat','outcome']
    
    return df_true

In [76]:
'''

create synthetic data with:
    3 numerical features (Gaussian), 1 categorical (sensitive attribute) 
    logistic outcome model s.t. outcome = Indicator[logit(effect_param*features) >= 0.5]
    
create minority/majority groups according to r param

simulate Bayes Optimal Classifiers for minority and majority

flip labels according to eta param

ensure equal base rates (proportion of positive examples) across both groups

'''

def true_label_generation(r, eta, n):

    ''' 
    delete this variable to allow user to control percentage of positively labeled examples
    eg: let outcome_continuous >= 0.2 implies 80% positively labeled samples
    '''
    effect_param = [0.5, -0.2, 0.1] # causal effect parameter (to create outcomes)

    # required: len(cat_probabilities) = n_cat_features
    n_cat_features = 2
    cat_probabilities = [0.5, 0.5] 

    # numerical feature params
    num_feature_mean = [0, 0, 0]
    num_feature_cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

    # features
    num_features = np.random.multivariate_normal(num_feature_mean, num_feature_cov, n)
    cat_features = get_cat_features(r=r, n=n)

    # outcomes
    outcome_continuous = 1/(1+np.exp(-np.matmul(num_features,effect_param))) # logit model + no added noise
    outcome_binary = np.where(outcome_continuous >= 0.5 , 1, 0).reshape(n,1)
    
    df_synthetic = pd.DataFrame(pd.DataFrame(np.hstack((num_features,
                                                        cat_features, outcome_binary))))
    df_synthetic.columns = ['num1','num2','num3','cat','outcome']
    
    df_majority = df_synthetic[df_synthetic['cat'] == 1]
    df_minority = df_synthetic[df_synthetic['cat'] == 0]
    
    df_synthetic = get_bayes_optimal_labels(df_majority, df_minority)
    
    df_synthetic = flip_labels(df_synthetic, eta)
    
    df_majority = df_synthetic[df_synthetic['cat'] == 1]
    df_minority = df_synthetic[df_synthetic['cat'] == 0]
    
    df_synthetic = equal_base_rates(df_majority, df_minority)
    
    return df_synthetic 

df_synthetic = true_label_generation(r=1/3, eta=0.25, n=2000)

# Preparation

### Data Preprocessing

In [77]:
# split into train and test
df_train = df_synthetic.loc[range(0,int(len(df_synthetic)/2)), :]
df_test = df_synthetic.loc[range(int(len(df_synthetic)/2), len(df_synthetic)), :]

# format data
X_true = df_test.iloc[:, :-1].values
y_true = df_test.iloc[:, -1].values

sens_attrs_true = [df_train['cat']]

# Bias Injection

In [78]:
def under_sample(df_minority_positive, beta):
    X_min = df_minority_positive.iloc[:, :].values
    
    # keep each example with probability beta
    for i in range(len(X_min)):
        if random.uniform(0,1) > beta:
            X_min = np.delete(X_min, 0, axis=0)
    
    df_minority_positive = pd.DataFrame(pd.DataFrame(X_min))
    df_minority_positive.columns = ['num1','num2','num3','cat','outcome']
    return df_minority_positive

def get_biased_data(df_train, beta):
    df_majority = df_train[df_train['cat'] == 1]
    df_minority = df_train[df_train['cat'] == 0]
    
    # unfavored group with negative label
    df_minority_negative = df_minority[df_minority['outcome'] == 0.0]

    # unfavored group with positive label (preferred)
    df_minority_positive = df_minority[df_minority['outcome'] == 1.0]
    
    # data frame without positively labeled examples from minority class
    df_total = pd.concat([df_majority, df_minority_negative])
    
    # under-sampling process
    df_undersampled = under_sample(df_minority_positive, beta)

    # combine undersampled and original favored class to create dataset
    df_concat = pd.concat([df_total,df_undersampled])
    
    return df_concat

df_concat = get_biased_data(df_train, 0.5)

In [86]:
# for fairness measures later
df_sens = df_concat['cat']

# format data
X_bias = df_concat.iloc[:, :-1].values
y_bias = df_concat.iloc[:, -1].values

# Model

### Model Selection + Training (TODO: modularize)

In [80]:
# modularize and add data struct of different ml techniques
classifier = LogisticRegression(random_state=42)

# Synthetic Data
classifier_true = classifier.fit(X_true, y_true)
y_pred_truth = classifier_true.predict(X_true)

classifier_bias = classifier.fit(X_bias, y_bias)
y_pred_bias = classifier_bias.predict(X_bias)
y_pred_bias_on_true = classifier_bias.predict(X_true)

### Model Performance (TODO: modularize)

In [81]:
print("Synthetic Data\n")

print("Accuracy of Ground Truth Model on Ground Truth Data: ", accuracy_score(y_pred_truth, y_true))
print("Accuracy of Biased Model on Biased Data: ", accuracy_score(y_pred_bias, y_bias))
print("Accuracy of Biased Model on Ground Truth Data: ", accuracy_score(y_pred_bias_on_true, y_true))

Synthetic Data

Accuracy of Ground Truth Model on Ground Truth Data:  0.802
Accuracy of Biased Model on Biased Data:  0.8358038768529077
Accuracy of Biased Model on Ground Truth Data:  0.817


In [83]:
# Ground Truth Model on Ground Truth Data

fpr_true = MetricFrame(false_positive_rate, y_true, y_pred_truth, sensitive_features = sens_attrs_true[0])
print("Overall Accuracy: ", fpr_true.overall)
print("Group Accuracy : ", fpr_true.by_group)

print("\n")
fnr_true = MetricFrame(true_positive_rate, y_true, y_pred_truth, sensitive_features = sens_attrs_true[0])
print("Overall True Positive Rate: ", fnr_true.overall)
print("Group True Positive Rate : ", fnr_true.by_group)

Overall Accuracy:  0.30851063829787234
Group Accuracy :  cat
0.0    0.279661
1.0    0.321705
Name: false_positive_rate, dtype: object


Overall True Positive Rate:  0.8685897435897436
Group True Positive Rate :  cat
0.0    0.846154
1.0    0.882051
Name: true_positive_rate, dtype: object


# Fairness Intervention

In [84]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
np.random.seed(0)

#### Synthetic Data

In [85]:
constraint = EqualizedOdds()
mitigator_true = ExponentiatedGradient(classifier_true, constraint)
mitigator_true.fit(X_true, y_true, sensitive_features = sens_attrs_true[0])
y_pred_mitigated_true = mitigator_true.predict(X_true)

In [88]:
constraint = EqualizedOdds()
mitigator_bias = ExponentiatedGradient(classifier_bias, constraint)
mitigator_bias.fit(X_bias, y_bias, sensitive_features = df_sens)
y_pred_mitigated_bias = mitigator_bias.predict(X_bias)
y_pred_mitigated_bias_on_true = mitigator_bias.predict(X_true)

# Evaluation

In [89]:
print("Synthetic Data\n")

print("Accuracy of Ground Truth Model + Fairness Intervention on Ground Truth Data: ",
      accuracy_score(y_pred_mitigated_true, y_true))

print("Accuracy of Biased Model + Fairness Intervention on Ground Truth Data: ",
      accuracy_score(y_pred_mitigated_bias_on_true, y_true))

Synthetic Data

Accuracy of Ground Truth Model + Fairness Intervention on Ground Truth Data:  0.757
Accuracy of Biased Model + Fairness Intervention on Ground Truth Data:  0.812


### Bias vs Accuracy vs Fairness Trade-Off

In [ ]:
# if verbose, shows "Finished iteration: ... "
# if apply_fairness, uses fairness intervention
def tradeoff_visualization(classifier, X_true, y_true, df_favored,
                           df_unfavored_positive, df_unfavored_negative,
                           sensitive_feature = "cat",
                           label = "outcome", cat_cols = [],
                           apply_fairness = False, verbose = False):
    
    bias_amts = list(range(0,30))
    accuracy_on_true = []
    accuracy_on_biased = []
    accuracy_on_true_mitigated = []
    accuracy_on_biased_mitigated = []
    
    dataset_size_true = np.full(shape=len(bias_amts), fill_value= X_true.shape[0]).tolist()
    dataset_size_bias = []

    df_undersampled = df_unfavored_positive.sample(n=len(df_unfavored_positive), random_state=42)

    for i in range(30):
        # under-sampling process
        if i != 0:
            df_undersampled = df_undersampled.sample(n=int(len(df_undersampled)*0.9), random_state=42)

        # combine undersampled and original favored class to create dataset
        df_concat = pd.concat([df_favored, df_unfavored_negative, df_undersampled])
        df_sens = df_concat[sensitive_feature]

        # format data
        X_bias = df_concat.iloc[:, :-2].values
        y_bias = df_concat.iloc[:, -1].values

        X_bias_true = df_concat.iloc[:, :-1].values
        y_bias_true = df_concat.iloc[:, -1].values

        dataset_size_bias.append(X_bias_true.shape[0])
        classifier_bias = classifier.fit(X_bias_true, y_bias_true)
        
        if apply_fairness:
            constraint = EqualizedOdds()
            classifier_mitigated_bias = ExponentiatedGradient(classifier_bias, constraint)
            classifier_mitigated_bias.fit(X_bias_true, y_bias_true, sensitive_features = df_sens)
            
            # testing on biased data WITH fairness intervention
            y_pred_mitigated_bias = classifier_mitigated_bias.predict(X_bias_true)
            
            # testing on GT data WITH fairness intervention
            y_pred_mitigated_bias_on_true = classifier_mitigated_bias.predict(X_true)
        
        # testing on biased data withOUT fairness intervention
        y_pred_bias = classifier_bias.predict(X_bias_true)
        
        # testing on GT data withOUT fairness intervention
        y_pred_bias_on_true = classifier_bias.predict(X_true)

        # model performance
        
        if apply_fairness:
            # on biased data
            acc_bias_mitigated = accuracy_score(y_pred=y_pred_mitigated_bias, y_true=y_bias_true)
            accuracy_on_biased_mitigated.append(acc_bias_mitigated)
            
            # on GT data
            acc_bias_mitigated_on_true = accuracy_score(y_pred=y_pred_mitigated_bias_on_true, y_true=y_true)
            accuracy_on_true_mitigated.append(acc_bias_mitigated_on_true)
        
        # on biased data
        acc_bias = accuracy_score(y_pred=y_pred_bias, y_true=y_bias_true)
        accuracy_on_biased.append(acc_bias)
        # on GT data
        acc_bias_on_true = accuracy_score(y_pred=y_pred_bias_on_true, y_true=y_true)
        accuracy_on_true.append(acc_bias_on_true)
        
        '''
        # fairness performance (TODO)
        eod_true = equalized_odds_difference(y_true=y_bias_true, y_pred = y_pred_bias, sensitive_features=df_sens)
        eod_on_true.append(eod_true)

        eod_bias_on_true = equalized_odds_difference(y_true=y_true,\
        y_pred = y_pred_bias_on_true, sensitive_features=sens_attrs[1])
        eod_on_biased.append(eod_bias_on_true)

        # table visualization 
        table_elem = [i*10, acc_bias, acc_bias_on_true]
        table.append(table_elem)
        '''
        
        if verbose:
            print("Finished Iteration: ", i)

    return bias_amts, dataset_size_true, dataset_size_bias, accuracy_on_biased,\
           accuracy_on_true, accuracy_on_biased_mitigated, accuracy_on_true_mitigated

In [ ]:
def accuracy_visualizations(bias_amts, dataset_size_true, dataset_size_bias,
                            accuracy_on_biased = [], accuracy_on_true = [],
                            accuracy_on_biased_mitigated = [],
                            accuracy_on_true_mitigated = [], fairness = False):
    
    if fairness:
        plt.figure(figsize=(17,7))

        plt.subplot(1,2,1)
        plt.plot(bias_amts, accuracy_on_true_mitigated, label = 'Ground Truth')
        plt.plot(bias_amts, accuracy_on_biased_mitigated, label = 'Biased Data')
        plt.xlabel("Number of iterations of removing 10% of positively labeled minority samples")
        plt.ylabel("Accuracy Score")
        plt.title("Biased Model Accuracy")
        #plt.ylim(0.97, 0.99)
        plt.legend()

        plt.subplot(1,2,2)
        plt.plot(bias_amts, dataset_size_true, label = 'Ground Truth')
        plt.plot(bias_amts, dataset_size_bias, label = 'Biased Data')
        plt.xlabel("Number of iterations of removing 10% of positively labeled minority samples")
        plt.ylabel("Dataset Size")
        plt.legend()

        plt.show()
        
    else:
        plt.figure(figsize=(17,7))

        plt.subplot(1,2,1)
        plt.plot(bias_amts, accuracy_on_true, label = 'Ground Truth')
        plt.plot(bias_amts, accuracy_on_biased, label = 'Biased Data')
        plt.xlabel("Number of iterations of removing 10% of positively labeled minority samples")
        plt.ylabel("Accuracy Score")
        plt.title("Biased Model Accuracy")
        #plt.ylim(0.97, 0.99)
        plt.legend()

        plt.subplot(1,2,2)
        plt.plot(bias_amts, dataset_size_true, label = 'Ground Truth')
        plt.plot(bias_amts, dataset_size_bias, label = 'Biased Data')
        plt.xlabel("Number of iterations of removing 10% of positively labeled minority samples")
        plt.ylabel("Dataset Size")
        plt.legend()

        plt.show()

In [ ]:
def total_visualizations(bias_amts, accuracy_on_biased, accuracy_on_true,
                        accuracy_on_biased_mitigated, accuracy_on_true_mitigated,
                        dataset_size_true, dataset_size_bias):
    plt.figure(figsize=(17,7))

    plt.subplot(1,2,1)
    plt.plot(bias_amts, accuracy_on_biased, label = 'Tested On Biased Data + No Fairness Intervention', color = "red")
    plt.plot(bias_amts, accuracy_on_biased_mitigated, label = 'Tested On Biased Data + Fairness Intervention', color = "green")
    plt.plot(bias_amts, accuracy_on_true, label = 'Tested On Ground Truth + No Fairness Intervention', color = "blue")
    plt.plot(bias_amts, accuracy_on_true_mitigated, label = 'Tested On Ground Truth + Fairness Intervention', color = "purple")
    plt.xlabel("Number of iterations of removing 10% of positively labeled minority samples")
    plt.ylabel("Accuracy Score")
    #plt.axhline(y=accuracy_score(y_pred_truth, y_true), color = "green", label = "Ground Truth Model On Ground Truth Data", alpha = 0.5)
    plt.title("Accuracy of Biased Model (trained on biased data)")
    plt.legend(loc = 3)

    plt.subplot(1,2,2)
    plt.plot(bias_amts, dataset_size_true, label = 'Ground Truth')
    plt.plot(bias_amts, dataset_size_bias, label = 'Biased Data')
    plt.xlabel("Number of iterations of removing 10% of positively labeled minority samples")
    plt.ylabel("Dataset Size")
    plt.title("Amount of Minority Samples Removed vs Dataset Size")
    plt.legend()

    plt.show()

In [ ]:
classifier = LogisticRegression()

bias_amts, dataset_size_true, dataset_size_bias, \
accuracy_on_biased, accuracy_on_true, accuracy_on_biased_mitigated, accuracy_on_true_mitigated = \
tradeoff_visualization(classifier, X_true=X_syn_true, y_true=y_syn_true, \
                       df_favored=df_favored_syn, df_unfavored_positive= df_unfavored_syn_positive,\
                       df_unfavored_negative= df_unfavored_syn_negative, sensitive_feature="cat",
                       cat_cols=[3], label = "outcome",
                       apply_fairness=True,verbose=True)

In [ ]:
# without fairness intervention
accuracy_visualizations(bias_amts, dataset_size_true, dataset_size_bias, accuracy_on_biased, accuracy_on_true, accuracy_on_biased_mitigated, accuracy_on_true_mitigated, False)

In [ ]:
# with fairness intervention
accuracy_visualizations(bias_amts, dataset_size_true, dataset_size_bias, accuracy_on_biased, accuracy_on_true, accuracy_on_biased_mitigated, accuracy_on_true_mitigated, True)

In [ ]:
total_visualizations(bias_amts, accuracy_on_biased, accuracy_on_true,
                    accuracy_on_biased_mitigated, accuracy_on_true_mitigated,
                    dataset_size_true, dataset_size_bias)